## 模型API
- 查阅时应该找文档，这个只是为了减慢阅读速度的笔记

### Sequential模型和函数式模型的通用API

In [ ]:
model.summary()#打印模型的概况
model.get_config()#返回包含模型配置信息的Python字典。

In [ ]:
#模型也可以从它的config信息中重构回去
config = model.get_config()
model = Model.from_config(config)
#sequential独有：
model = Sequential.from_config(config)

In [ ]:
model.get_layer() #根据层名或下标获得层对象
model.det_weights()  #模型权重列表，类型为np.ndarray
model.set_weights()  #从np.ndarray中载入权重--形状匹配
model.to_json()      #返回代表模型的JSON字符串，仅包含网络结构，不包含权值。
model.to_yaml()       #和json类似

In [ ]:
#可以从JSON字符串中重构原模型：
from models import model_from_json

json_string = model.to_json()
model = model_from_json(json_string)

#yaml也是一样的
from models import model_from_yaml

yaml_string = model.to_yaml()
model = model_from_yaml(yaml_string)

In [ ]:
model.save_weights(filepath) #保存权重文件，类型是HDF5(.h5)

In [ ]:
"""
从HDF5文件中加载权重到当前模型中, 默认情况下模型的结构将保持不变。
如果想将权重载入不同的模型（有些层相同）中，则设置by_name=True，
只有名字匹配的层才会载入权重
"""
model.load_weights(filepath, by_name=False)

### Sequential模型API

In [ ]:
model.layers  #获得模型中层的list

### 函数式模型
- [Link](http://keras-cn.readthedocs.io/en/latest/models/model/)

In [1]:
from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

Using TensorFlow backend.


#### 属性

In [4]:
print(model.layers)
print(model.inputs)
print(model.outputs)

[<keras.engine.topology.InputLayer object at 0x000001C94C9CA710>, <keras.layers.core.Dense object at 0x000001C94C9CA4E0>]
[<tf.Tensor 'input_1:0' shape=(?, 32) dtype=float32>]
[<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 32) dtype=float32>]


#### 方法

##### [compile]()：编译模型以提供训练，只是predict的话，不需要
- 总体：
```python
compile(self, optimizer, loss, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
```
- 常见参数：
    - optimozer
    - loss
    - metrics 评估指标列表
        - 如果要在多输出模型中为**不同的输出指定不同的指标**，可像该参数传递一个字典，例如metrics={'ouput_a': 'accuracy'}

##### [fit]()：训练模型
- 总体：
```python
fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
```
- 常见参数：
    - X
    - y
    - batch_size
    - epochs
    - verbose:日志
        - 0：不输出
        - 1：输出进度记录条
        - 2：输出每个epoch的记录--其他函数大多只能取0/1
    - validation_split:0~1之间的浮点数，用来指定训练集的一定比例数据作为验证集。验证集将不参与训练，并在每个epoch结束后测试的模型的指标，如损失函数、精确度等。
        - 注意，validation_split的划分在shuffle之后，因此如果你的数据本身是有序的，需要先手工打乱再指定validation_split，否则可能会出现验证集样本不均匀。
    - validation_data:(X, y),会覆盖validation_split
    - shuffle:**每个epoch**前随机打乱输入样本的顺序。

##### [evalute]():按batch计算给定数据的模型误差
- 总体：
```python
evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None)
```
- 参数：
    - X
    - Y
    - batch_size:
    - ver_bose:只能取0/1

##### [predict]():
- 返回的类型是ndarray
```python
predict(self, x, batch_size=32, verbose=0)
```

##### [xx_on_batch]()
- train/test/predict

## 损失函数

- 可以直接使用自己定义的函数名，或者是Tensorflow的符号函数
- 常用的目标函数：
```python
from keras import losses
loss = losses.xxx
```
    - mean_squared_error
    - mean_absolute_error
    - squared_hinge
    - categorical_crossentropy
    ...
- **注意：**在使用categorical_crossentropy作为损失函数时，对应类别应该为多类别，label应该使用one_hot编码，可以使用to_categorical直接转换

## Optimzer

In [ ]:
from keraas import optimzers

sgd = optimzers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimzer=sgd)

##### 梯度剪裁
```python
from keras import optimizers

#All parameters gradients will be clipped to
#a maximun of 1.
sgd = optimzers.SGD(lr=0.001, clipnorm=1.)

#...
# a maximum value of 0.5 and
# a minimum value of -0.5.
sgd = optimzer.SGD(lr=0.1, clipvalue=0.5)
```

### 常用optimzer

#### TF优化器的包装器
```python
keras.optimizers.TFOptimizer(optimizer)
```

#### SGD
```python
keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
```
- 参数
    - lr：
    - momentum
    - decay
    - nesterov:True or False

#### RMSprop
```python
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
```
- 参数
    - lr
    - rho
    - epsilon：防止除0的参数

#### Adagrad
```python
keras.optimizers.Adagrad(lr=0.01, epsilon=1e-06)
```

#### 目前没用过的 Adadelte
```python
keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)
```

#### Adam
```python
keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
```

##### Adamax
- Adam的变体--在Adam论文里
```python
keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
```

#### Nadam
- Nesterov Adam optimizer: Adam本质上像是带有动量项的RMSprop，Nadam就是带有Nesterov 动量的Adam RMSprop
```python
keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
```

## 激活函数
- 可以单独设置激活层或者在构造层的时候加入，或者用TF的函数
```python
#Activation层
keras.layers.core.Activation(activation)
```
- 预定义的激活函数(知道的)
    - softplus
    - relu
    - softmax
    - tanh
- 高级激活函数
    - LeaktReLU:避免ReLU可能出现的失活现象
```python
keras.layers.advanced_activations.LeakyReLU(alpha=0.3)
```
    - PReLU:参数化的ReLU,alpha可学习
```python
keras.layers.advanced_activations.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)
```
        - 参数：
            - alpha_constraint:**约束项？？**
            - shared_axes:共享参数的轴，比如输入tensor为`(batch_size,height,width,channels)`,如果希望每个filter都用相同的参数，只需要设置 `hared_axes=[1, 2]`
    - ELU层：指数线性单元
    ```python
    keras.layers.advanced_activations.ELU(alpha=1.0)
    ```
        - 表达式
```python
f(x) = alpha * (exp(x) - 1.) for x < 0, f(x) = x for x>=0
```
    - ThresholdedReLU层
    ```python
    keras.layers.advanced_activations.ThresholdedReLU(theta=1.0)
    ```
        - 表达式
        ```ptrhon
        f(x) = x for x > theta,f(x) = 0 otherwise
        ```

In [ ]:
from keras import backend as K
#使用TF自带的函数作为激活函数
def tanh(x):
    return K.tanh(x)

X = Dense(64, activation=tanh)(X)

## 性能评估

##### 在上面有提过，模型编译时，通过metrics进行设置


In [ ]:
from keras import metrics


model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['mae', 'acc'])

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=[metrics.mae, metrics.categorical_accuracy])


##### 自定义函数
- 输入
    > y_true  
     y_pred
- 返回
    > 返回单个张量,或从`metric_name`映射到`metric_value`的字典

In [ ]:
import keras.backend as K

def mean_err(y_true, y_pred):
    return K.mean(y_pred -  y_true)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['mean_err', mean_pred])

## 初始化方法
- 正常对应设置中的`kernel_initializer`或者`bias_initializer`
- 使用对应初始化方法的字符串或者函数名
- 自定义估计目前没啥用

### Initializer类
- Initializer是所有初始化方法的父类，不能直接使用，如果想要定义自己的初始化方法，需要继承此类
- 常见方法：
```python
keras.initializers.Ones()
keras.initializers.Zeros()
keras.initializers.Constant(value=0)
#正态分布
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None))
#均匀分布
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)
#截尾正态分布:位于均值两边两个stddev外的数据会直接重新生成
keras.initializers.TruncatedNormal(mean=.0, stddev=0.005, seed=None)
#xavier初始化
'''
参数从[-limit, limit]范围内的均匀分布产生，limit=sqrt(6/fan_in+fan_out),fan_in,fan_out分别是输入，输出的权重张量的单元数
'''
keras.initializers.glorot_uniform(seed=None)
#...还有很多初始化方法
```

### 正则项
- [原码](https://github.com/keras-team/keras/blob/master/keras/regularizers.py·)
- 在层定义的时候添加的项-->会作为损失函数的一部分进行优化
- 都是keras.regularizer.Regularizer对象，在层中添加的参数
    - `kernel_regularizer`:权重惩罚
    - `bias_regularizer`：偏置惩罚
    - `activity_regulaizer`:输出的惩罚
- 可用的正则项
    ```python
    keras.regularizers.l1(0.)  
    keras.regularizers.l2(0.)  
    keras.regularizers.l1_l2(l1=0., l2=0.)  
    ```
- 自定义的正则项
```python
    from keras import backend as K

    def l1_reg(weight_matrix):
        return 0.01*K.sum(K.abs(weight_matrix))
    X = Dense(64, input_dim=64, kernel_regularizer=l1_reg)(X)
```

### 约束项
- 在constraints模块中
- 构造中常见的参数
     - kernel_constraint
     - bias_constraint
- 预定义的约束项
     - max_norm(m=2)：模值约束
     - non_neg()：非负约束
     - unit_norm()：**？？单位范数约束，强制矩阵沿最后一个轴拥有单位范数**
     - min_max_norm(min_value=0., max_value=1., rate=1., axix=0):最大最小范数约束

In [ ]:
from keras.Constrains import maxnorm

X = Dense(64, kernel_constraint=max_norm(2.))

### 回调函数Callbacks
- 在训练的特定阶段被调用的函数集，可以比如用于观察训练中网络内部的变量观察这类的(Tensorboard之类的)
- 一个类--`keras.callbacks.Callback`，定义新的回调函数需要继承这个类

In [ ]:
#例子:记录损失函数的历史数据
from keras.models import Model
from keras.utils import to_categorical

class LossHistory(keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
X_i = Input(input_shape)
X = Dense(10, input_dim=784, kernel_initializer='uniform')(X_i)
Y = Activation('softmax')(X)
model = Model(inputs=X_i, outputs=Y)
model.compile(loss='categorical_crossentropy', optimzer='rmsprop')
history = LossHistory()
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, callbacks=[history])
print(history.losses)

##### 回调函数以logs为字典，在训练的不同阶段会记录不同的数据
- on_epoch_end:`log={acc, loss}`，如果指定了验证集，还会包含`val_acc和val_loss`,**注意这里要求compile中`metrics`指定`accuracy`**
- on_batch_begin:`size`，当前batch的样本数
- on_batch_end:`logs={loss, (acc)}`

##### 几个常用的callback函数  
1. 在每个epoch后面保存模型到`filepath`
```python
keras.callbacks.ModelCheckpoint(  
    filepath,monitor='val_loss',  
    verbose=0, save_best_only=False,save_weights_only=False,  
    mode='auto', period=1)```
    - 这里`filepath`可以是格式化的字符串，里面的占位符会被`epoch`和传入`on_epoch_end`的`logs`的关键字替代，比如 
```python
# 生成对应epoch和loss的文件
filepath = 'weights.{epoch:02d-{val_loss:.3f}}.hdf5'
```
    - `monitor`为需要监视的变量名
    - `save_best_only`:为`True`时，只保存在验证集上性能最好的模型
    - `mode`:在`save_best_only=True`时决定性能最佳模型的评判准则，例如，当监测值为`val_acc`时，模式应为`max`，当检测值为`val_loss`时，模式应为`min`,在`auto`模式下，评价准则由被监测值的名字自动推断
    - `save_weight_only`:`False`时将保存模型结构，配置信息等
    - `period`:间隔几个`epoch`执行一次  
2. 早停策略
```python
keras.callbacks.EarlyStopping(  
    monitor='val_loss', patience=0, verbose=0, mode='auto')
```
    - `monitor`:需要监视的量
    - `patience`:当early stop被激活，则经过patience个epoch后停止训练
    - 'mode':‘auto’，‘min’，‘max’之一，在`min`模式下，如果检测值停止下降则中止训练。在`max`模式下，当检测值不再上升则停止训练
- 可视化的`Tensorboard`
```python
keras.callbacks.TensorBoard(  
    log_dir='./logs', histogram_freq=0,  
    write_graph=True,write_images=False, embeddings_freq=0,   
    embeddings_layer_names=None, embeddings_metadata=None)
```
    - `log_dir`:保存日志文件的地址
    - `histogram_freq`:多少个epoch计算一次
3. 创建简单的callback类
```python
keras.callbacks.LambdaCallback(  
    on_epoch_begin=None, on_epoch_end=None,  
    on_batch_begin=None, on_batch_end=None,  
    on_train_begin=None, on_train_end=None)
```
    - 匿名函数的参数：
        - `on_epoch_begin`,`on_epoch_end`:假定输入是`eppoch`，`logs`
        - `on_batch_begin`,`on_batch_end`:假定输入时`batch`，`logs`
        - `on_train_begin`,`on_train_end`:假定输入时`logs`
    - 示例：

In [ ]:
#打印batch number
batch_print_callback = LambdaCallback(
    on_batch_begin=lambda: batch, logs:print(batch))

#画出loss
import numpy as np
import matplotlib.pyplot as plt

plot_loss_callback = LambdeCallback(
    on_epoch_end=lambda epoch, logs:plt.plot(np.arange(epoch), 
                                             logs['loss']))

In [ ]:
#保存模型
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Dense(10, input_dim=784, kernel_initializer='uniform'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
'''
saves the model weights after each epoch if the validation loss decreased
'''
checkpointer = ModelCheckpoint(filepath="/tmp/weights.hdf5", verbose=1, save_best_only=True)
model.fit(X_train, Y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])

### 预训练模型
> 提供常见的图像分类模型  
 [权重文件](https://pan.baidu.com/s/1geHmOpH#list/path=%2F&parentPath=%2F)
![](images/keras_nets.png)
< 下面给一些迁移学习的例子，keras提供的模型可以直接在文档查找

In [ ]:
#ResNet50做分类
from keras.applications.resnet50 import ResNet50
from keras.preprossing import image
from keras.applications.resnet50 import prepross_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')
img_path = 'xx.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)#batch轴
x = preprocess_input(x)

preds = model.predict()
classes, description, probabiity = decode_predictions(preds, top=3)
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (
#u'n01871265', u'tusker', 0.1122357), 
#(u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
#VGG16特征提取
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)
img_path = 'xx.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
#VGG-19提取中间层特征
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [ ]:
#在新的类别上fine_tune inceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

base_model = InceptionV3(weights='imagenet', include_top=False)#不需要顶层
x = base_model.output
#添加需要的层
x = GlobalAveragePooling2D()(X)
perd = Dense(200, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=pred)
#原模型不再训练,先训练最后两层
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(opptimizer='sgd', loss='categorical_crossentropy')
model.fit_generator(...)
#fine_tune模型，依旧选择需要的层，设置trainable
for i,layer in enumerate(base_model.layers):
    print(i, layer.name)
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.01, momentum0.9), loss='categorical_crossentropy')
model.fit(...)

In [ ]:
#构建新的输入的InceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

### 常用数据集
```python
from keras.datasets import ...
```

### 可视化模型
- 个人还是喜欢tensorboard

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')
'''
参数：
    show_shapes:显示输出数据的形状，默认False
    show_layer_names:是否显示层名称
'''

In [ ]:
#在ipython显示
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))
#依赖 pydot-ng 和 graphviz
#pip install pydot-ng & brew install graphviz